In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import time
import random
from datetime import datetime
from GoogleNews import GoogleNews
from serpapi import GoogleSearch
import pandas as pd
from openai import OpenAI
import json

# ================================================================
# Keys
# ================================================================
OPENAI_API_KEY = 
SerpAPI_tkn1 = 

client = OpenAI(api_key=OPENAI_API_KEY)


In [4]:
# ================================================================
# 🔍 ANÁLISIS INDIVIDUAL DE UNA NOTICIA (con exportación a CSV)
# ================================================================
import requests, json, io, pandas as pd, time
from bs4 import BeautifulSoup
from datetime import datetime

# ================================================================
# 🤖 Funciones base
# ================================================================
def obtener_texto_completo(url):
    """Descarga y limpia el texto principal de una noticia."""
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")
        for s in soup(["script", "style", "noscript", "header", "footer", "aside", "form"]):
            s.extract()
        texto = " ".join(soup.stripped_strings)
        return texto[:8000]
    except Exception as e:
        return f"Error al obtener contenido: {e}"


def analizar_con_ia(url, client, titulo_manual="Noticia sin título", descripcion_manual=""):
    """
    Analiza una sola noticia con GPT-4o-mini y devuelve un JSON estructurado.
    """
    print(f"\n🧠 Analizando: {url}")

    contenido_completo = obtener_texto_completo(url)
    texto_analizar = (
        contenido_completo
        if "Error" not in contenido_completo
        else f"Título: {titulo_manual}\nDescripción: {descripcion_manual}"
    )

    prompt = f"""
Analiza la siguiente noticia en español sobre educación, sindicatos o protestas de docentes.

CRITERIOS ESTRICTOS:
- Un **paro docente verdadero** solo existe si hubo suspensión de clases programadas.
- Si fue marcha, manifestación o concentración SIN suspensión de clases, NO es paro.

Devuelve un JSON con estas claves:

- "es_paro_docente": true/false → true solo si hubo suspensión de clases
- "justificacion_paro": texto breve (2-4 líneas) explicando por qué es o no un paro
- "organizaciones_sindicales": lista de sindicatos mencionados u organizaciones políticas o partidos políticos (ej: ["FECODE", "ADE"]), o []
- "hay_suspension_clases": true/false → si se suspendieron clases
- "duracion_dias": número de días estimados, o null
- "razones_paro": resumen de las demandas principales
- "ubicacion_bogota": true/false → si ocurre en Bogotá/Cundinamarca
- "ubicacion_Colombia": true/false → si el paro abarca el territorio Nacional
- "costo_mencionado": monto económico si se menciona, o null
- "resumen": breve resumen (máx. 3 líneas)
- "fecha_paro": ¿cuál es la fecha exacta del paro (día, mes, año)? 
- "tipo_movilizacion": ¿cuál es el tipo de movilización, paro docente con las caracteristicas que ya vimos, paro de otro gremio, demostración, plantón, toma, marcha pacífica, etc?
- "ubicacion": además del país y ciudad, podemos identificar localidades? barrios? calles? edificios? colegios específicos o barrios?
- "fuente_oficial": ¿es una fuente oficial de algún sindicato, secretaria de educacion o es medio de comunicación? 

Devuelve SOLO el JSON válido, sin texto extra.

Texto a analizar:
{texto_analizar}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0.1,
            messages=[{"role": "user", "content": prompt}],
        )
        respuesta_texto = response.choices[0].message.content.strip()

        # Intentar parsear JSON
        try:
            analisis = json.loads(respuesta_texto)
        except json.JSONDecodeError:
            if "```json" in respuesta_texto:
                inicio = respuesta_texto.find("```json") + 7
                fin = respuesta_texto.find("```", inicio)
                analisis = json.loads(respuesta_texto[inicio:fin].strip())
            else:
                print("⚠️ No se pudo parsear el JSON, usando valores por defecto")
                analisis = {
                    'es_paro_docente': False,
                    'justificacion_paro': 'Error al analizar',
                    'organizaciones_sindicales': [],
                    'hay_suspension_clases': False,
                    'duracion_dias': None,
                    'razones_paro': '',
                    'ubicacion_bogota': False,
                    'ubicacion_Colombia': False,
                    'costo_mencionado': None,
                    'resumen': '',
                    'fecha_paro': None,
                    'tipo_movilizacion': '',
                    'ubicacion': '',
                    'fuente_oficial': ''
                }

        return analisis

    except Exception as e:
        print(f"❌ Error en análisis IA: {e}")
        return {
            'es_paro_docente': False,
            'justificacion_paro': f'Error: {str(e)}',
            'organizaciones_sindicales': [],
            'hay_suspension_clases': False,
            'duracion_dias': None,
            'razones_paro': '',
            'ubicacion_bogota': False,
            'ubicacion_Colombia': False,
            'costo_mencionado': None,
            'resumen': '',
            'fecha_paro': None,
            'tipo_movilizacion': '',
            'ubicacion': '',
            'fuente_oficial': ''
        }


# ================================================================
# 🚀 Ejecución Principal
# ================================================================
if __name__ == "__main__":
    print("=" * 60)
    print("🔥 ANALIZANDO UNA SOLA NOTICIA")
    print("=" * 60)

    # 💬 Cambia esta URL por la que quieras analizar
    url = "https://www.elpais.com.co/colombia/paro-nacional-de-profesores-30-de-agosto-de-2023-fecha-hora-motivos-y-puntos-de-encuentro-de-las-manifestaciones-2907.html"

    # 🧠 Asegúrate de tener tu cliente:
    # from openai import OpenAI
    # client = OpenAI(api_key="TU_API_KEY_AQUI")

    analisis = analizar_con_ia(url, client)

    noticia = {
        'titulo': 'Noticia analizada individualmente',
        'fuente': 'Manual',
        'fecha_publicacion': '',
        'url': url,
        'descripcion': '',
        'periodo_busqueda': 'análisis_individual',
        'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        **analisis
    }

    df = pd.DataFrame([noticia])

    # ✅ Reordenar columnas
    columnas_importantes = [
        'titulo', 'fuente', 'fecha_publicacion', 'url',
        'es_paro_docente', 'hay_suspension_clases', 'ubicacion_bogota',
        'ubicacion_Colombia', 'duracion_dias', 'organizaciones_sindicales',
        'razones_paro', 'costo_mencionado', 'justificacion_paro', 'resumen',
        'fecha_paro', 'tipo_movilizacion', 'ubicacion',
        'descripcion', 'periodo_busqueda', 'fecha_extraccion', 'fuente_oficial'
    ]
    df = df[[col for col in columnas_importantes if col in df.columns]]

    # 💾 Guardar CSV directamente en memoria (para entornos de solo lectura)
    nombre_csv = "analisis_unico_noticia.csv"
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False, encoding="utf-8-sig")
    csv_data = csv_buffer.getvalue()

    # Mostrar salida CSV
    print("\n✅ Análisis completado. Primeras líneas del CSV generado:\n")
    print(csv_data[:800])

    # Si el entorno permite guardar archivos, intentar escribirlo
    try:
        with open(nombre_csv, "w", encoding="utf-8-sig") as f:
            f.write(csv_data)
        print(f"\n📁 Archivo guardado exitosamente como: {nombre_csv}")
    except OSError:
        print("\n⚠️ No se pudo guardar en disco (entorno de solo lectura). CSV disponible en memoria.")


🔥 ANALIZANDO UNA SOLA NOTICIA

🧠 Analizando: https://www.elpais.com.co/colombia/paro-nacional-de-profesores-30-de-agosto-de-2023-fecha-hora-motivos-y-puntos-de-encuentro-de-las-manifestaciones-2907.html

✅ Análisis completado. Primeras líneas del CSV generado:

titulo,fuente,fecha_publicacion,url,es_paro_docente,hay_suspension_clases,ubicacion_bogota,ubicacion_Colombia,duracion_dias,organizaciones_sindicales,razones_paro,costo_mencionado,justificacion_paro,resumen,fecha_paro,tipo_movilizacion,ubicacion,descripcion,periodo_busqueda,fecha_extraccion,fuente_oficial
Noticia analizada individualmente,Manual,,https://www.elpais.com.co/colombia/paro-nacional-de-profesores-30-de-agosto-de-2023-fecha-hora-motivos-y-puntos-de-encuentro-de-las-manifestaciones-2907.html,True,True,True,True,1,['FECODE'],Las demandas incluyen la mala prestación del servicio de salud del Magisterio y la oposición a dos proyectos de ley en el Congreso que afectan la educación.,,"El paro nacional de profesores implica 

In [3]:
df.to_csv("analisis_unico_noticia.csv", index=False, encoding="utf-8-sig")